# Reprojecting geographic data {#reproj-geo-data}

## Introduction

## Coordinate Reference Systems

## Querying and setting coordinate systems

## Geometry operations on projected and unprojected data

## When to reproject?

## Which CRS to use?

## Reprojecting vector geometries

## Reprojecting raster geometries

## Custom map projections

## Exercises